In [ ]:
! [ -d "PetImages" ] && echo "skipping" || (wget -nc --no-check-certificate https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip && unzip kagglecatsanddogs_3367a.zip "PetImages/*" -d . && rm kagglecatsanddogs_3367a.zip)
!rm -f "PetImages/Dog/11702.jpg" # Remove broken image
!rm -f "PetImages/Cat/666.jpg" # Remove broken image

In [ ]:
import os
import shutil

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
from sap_computer_vision.datasets import image_folder as imgf
import pathlib, json


dataset_folder = pathlib.Path('PetImages/')
datasets, class_names = imgf.split_and_register('dogs_vs_cats',
                                                base_dir=dataset_folder,
                                                splits={'test': 0.1, 'validation': 0.1, 'train': 0.8})

In [ ]:
from sap_computer_vision import setup_loggers, get_cfg, get_config_file
import numpy as np

out_dir = 'image_classification_model'
if pathlib.Path(out_dir).exists():
    raise RuntimeError('Result folder already exists. Please delete the folder or change the name of the output')

setup_loggers(out_dir)

cfg = get_cfg()
cfg.merge_from_file(get_config_file('Base-EarlyStopping'))
cfg.merge_from_file(get_config_file('Base-Evaluation'))
cfg.merge_from_file(get_config_file('ImageClassifier/Resnet50'))

cfg.OUTPUT_DIR = out_dir
cfg.DATASETS.TRAIN = ('dogs_vs_cats_train', )
cfg.DATASETS.TEST = ('dogs_vs_cats_validation', )

cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.BASE_LR = 0.001

cfg.SOLVER.GAMMA = float(np.sqrt(0.1))
cfg.SOLVER.EARLY_STOPPING.ENABLED = False

cfg.SOLVER.WARMUP_ITERS = max(int(0.01 * cfg.SOLVER.MAX_ITER), 50)
cfg.SOLVER.STEPS = [cfg.SOLVER.MAX_ITER * p for p in (0.05, 0.25, 0.375, 0.5, 0.75, 0.9)]
for aug in ['RANDOM_LIGHTING', 'RANDOM_BRIGHTNESS', 'RANDOM_SATURATION', 'RANDOM_CONTRAST', 'CROP']:
    if cfg.get(aug, None) is not None:
        cfg.INPUT[aug].ENABLED = True

cfg.SOLVER.IMS_PER_BATCH = 64
cfg.SOLVER.IMS_PER_BATCH_EVAL = 64

cfg.MODEL.IMAGE_CLASSIFIER.NUM_CLASSES = len(class_names)

cfg.TEST.EVAL_PERIOD = int(cfg.SOLVER.MAX_ITER * 0.25)

In [ ]:
from sap_computer_vision.engine import ImageClassificationTrainer

In [ ]:
trainer = ImageClassificationTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
cfg.DATASETS.TEST = ('dogs_vs_cats_test', )
metrics = trainer.test(cfg, trainer.model)
print(json.dumps(metrics, ident=2))